In [1]:
from __future__ import print_function
from PIL import Image
import cv2
import numpy as np
import math
import string
import sys
import os

In [2]:
count=0
part=0
rowNumber= 0
outputRowPath= '/Users/abhishekpandey/Desktop/stuff/MajorProject/main/segmentation/charWise/output'
inputRowPath= '/Users/abhishekpandey/Desktop/stuff/MajorProject/main/segmentation/wordWise/output/words'

In [3]:
for rowName in os.listdir(inputRowPath):
    
    if not rowName.startswith('.'):
        rowNumber= rowNumber+1
        rowDirectory= 'row'+ str(rowNumber)
        finalRowPath= os.path.join(inputRowPath, rowDirectory)
        wordNumber= 0
        for fileName in os.listdir(finalRowPath):
            img=cv2.imread(os.path.join(finalRowPath, fileName))
            if img is None:
                continue
            finalOutputRowPath= os.path.join(outputRowPath, rowDirectory)
            os.makedirs(finalOutputRowPath,exist_ok = True)
            
            wordNumber= wordNumber+1
            wordDirectory= 'word'+ str(wordNumber)
            wordPath= os.path.join(finalOutputRowPath, wordDirectory)
            os.makedirs(wordPath,exist_ok = True)
            
            
            rows,cols,w = img.shape

            gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            ret, threshold = cv2.threshold(gray_image,200,255,cv2.THRESH_BINARY)
            #dilating
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2000))
            dilation = cv2.dilate(threshold,kernel,iterations = 16)
            th2= dilation.copy()


            ret, threshold = cv2.threshold(th2,200,255,cv2.THRESH_BINARY_INV)
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2000))
            erosion = cv2.erode(threshold,kernel,iterations = 8)
            ret, threshold1 = cv2.threshold(erosion,200,255,cv2.THRESH_BINARY_INV)

            k= threshold1.copy()

            #defining the edges
            edges = cv2.Canny(k,50,150,apertureSize = 3)


            m=[]
            minLineLength = 100
            maxLineGap = 10
            lines = cv2.HoughLinesP(edges,1,np.pi/180,15,minLineLength,maxLineGap)
            for x in range(0, len(lines)):
                    for x1,y1,x2,y2 in lines[x]:
                            m.append(((x1,y1),(x2,y2)))


            sorted_m=sorted(m, key=lambda x: x[0][0])


            #drawing lines
            for i in range(len(sorted_m)):
                    cv2.line(img,sorted_m[i][0],sorted_m[i][1],(0,0,255),2)

            hough_lines= img.copy()
            p=[]
            sorted_m.append(((cols,rows),(cols,0)))

            size= (28, 28)

            count= 0
        
            for i in range(len(sorted_m)-1):
                p.append(hough_lines[sorted_m[i][1][1]:sorted_m[i][0][1],sorted_m[i][0][0]: sorted_m[i+1][0][0]])
                image = cv2.cvtColor(p[i], cv2.COLOR_BGR2GRAY)
                ret, o = cv2.threshold(image, 200, 255, cv2.THRESH_BINARY)
                if cv2.countNonZero(o) >0:
                    cv2.imwrite('/Users/abhishekpandey/Desktop/stuff/MajorProject/main/segmentation/charWise/verticalSegmention/Output_1_row_'+str(part)+'_'+'col_'+str(i)+'.jpg', o)

                    preProces_img= Image.open('/Users/abhishekpandey/Desktop/stuff/MajorProject/main/segmentation/charWise/verticalSegmention/Output_1_row_'+str(part)+'_'+'col_'+str(i)+'.jpg')

                    background_img= Image.open('/Users/abhishekpandey/Desktop/stuff/MajorProject/main/segmentation/charWise/black.png')
                    background_img.paste(preProces_img, (10,3))
                    box=(0,0,70,100)
                    cropped_image=background_img.crop(box)
                    resized_image=cropped_image.resize((28,28))
                    imageNumber= 'chatacter'+str(i)+'.jpg'
                    imagePath= os.path.join(wordPath, imageNumber)
                    resized_image.save(imagePath)
                   
            part=part+1




print("code complete")

code complete
